In [75]:
import pymongo
import json
import requests
import pandas as pd
import plotly.express as px

Dataset: https://www.kaggle.com/maximumbeings/usda-food-database-nutrients-json

In [76]:
#Command line insert json file
#mongoimport --db tutorial --collection nutrients --file ~/Downloads/nutrients.json
#to delete db: use db, db.dropDatabase()

In [77]:
from pymongo import MongoClient

In [78]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.tutorial
nutrients = db.nutrients

In [79]:
import pprint

In [80]:
#initialize empty lists
protein = []
fat = []
carbs = []
calories = []
group = []

In [81]:
#Upon examining the arrays, I found that index 0 of nutrients is always protein, 
#index 1 is fat, index 2 is carbs, index 4 is calories (unit kcal)

#Query
for item in nutrients.aggregate([{"$project": {"nutrients.abbr":1}},{"$limit":1}]):
    print(item)

{'_id': ObjectId('6222969911f4cc87dc547d8c'), 'nutrients': [{'abbr': 'PROCNT'}, {'abbr': 'FAT'}, {'abbr': 'CHOCDF'}, {'abbr': 'ASH'}, {'abbr': 'ENERC_KCAL'}, {'abbr': 'ALC'}, {'abbr': 'WATER'}, {'abbr': 'CAFFN'}, {'abbr': 'THEBRN'}, {'abbr': 'ENERC_KJ'}, {'abbr': 'SUGAR'}, {'abbr': 'FIBTG'}, {'abbr': 'CA'}, {'abbr': 'FE'}, {'abbr': 'MG'}, {'abbr': 'P'}, {'abbr': 'K'}, {'abbr': 'NA'}, {'abbr': 'ZN'}, {'abbr': 'CU'}, {'abbr': 'MN'}, {'abbr': 'SE'}, {'abbr': 'VITA_IU'}, {'abbr': 'RETOL'}, {'abbr': 'VITA_RAE'}, {'abbr': 'CARTB'}, {'abbr': 'CARTA'}, {'abbr': 'TOCPHA'}, {'abbr': 'VITD'}, {'abbr': 'CHOCAL'}, {'abbr': 'VITD'}, {'abbr': 'CRYPX'}, {'abbr': 'LYCPN'}, {'abbr': 'LUT+ZEA'}, {'abbr': 'VITC'}, {'abbr': 'THIA'}, {'abbr': 'RIBF'}, {'abbr': 'NIA'}, {'abbr': 'PANTAC'}, {'abbr': 'VITB6A'}, {'abbr': 'FOL'}, {'abbr': 'VITB12'}, {'abbr': 'CHOLN'}, {'abbr': 'VITK1'}, {'abbr': 'FOLAC'}, {'abbr': 'FOLFD'}, {'abbr': 'FOLDFE'}, {'abbr': 'TRP_G'}, {'abbr': 'THR_G'}, {'abbr': 'ILE_G'}, {'abbr': 'LEU

In [82]:
for item in nutrients.aggregate([{"$project": {"protein":{
        "$arrayElemAt":["$nutrients.value",0]}, "fat": {"$arrayElemAt":["$nutrients.value",1]},"carbs": {"$arrayElemAt":["$nutrients.value",2]},
                                               "calories": {"$arrayElemAt":["$nutrients.value",4]},
                                               "group":1}}]):
    group.append(item['group'])
    protein.append(item['protein'])
    fat.append(item['fat'])
    carbs.append(item['carbs'])
    calories.append(item['calories'])

In [83]:
df = pd.DataFrame(protein,columns=['protein'])
df['fat'] = fat
df['carbs'] = carbs
df['calories'] = calories
df['group'] = group

In [84]:
df

,protein,fat,carbs,calories,group
0,21.40,28.74,2.34,353.0,Dairy and Egg Products
1,0.28,99.48,0.00,876.0,Dairy and Egg Products
2,23.24,29.68,2.79,371.0,Dairy and Egg Products
3,0.85,81.11,0.06,717.0,Dairy and Egg Products
4,20.75,27.68,0.45,334.0,Dairy and Egg Products
...,...,...,...,...,...
8784,19.80,0.50,0.00,89.0,Finfish and Shellfish Products
8785,18.50,25.10,0.00,305.0,Finfish and Shellfish Products
8786,0.00,0.00,73.14,269.0,Sweets
8787,16.10,1.40,2.00,90.0,Finfish and Shellfish Products


In [85]:
macros_by_group = df.groupby('group').mean()

## Visualization

Average protein content among food groups. Which food group has the highest average protein content?

In [86]:
macros_by_group.reset_index(inplace=True)

In [87]:
macros_by_group

,group,protein,fat,carbs,calories
0,American Indian/Alaska Native Foods,15.996061,11.109758,11.253939,209.115152
1,Baby Foods,4.059864,5.455777,21.698011,151.179837
2,Baked Products,6.514642,14.222696,59.370785,234.524755
3,Beef Products,24.359105,12.235505,0.107253,211.622268
4,Beverages,2.235175,0.953261,17.428679,103.513208
5,Breakfast Cereals,8.320506,4.034017,73.308427,345.379213
6,Cereal Grains and Pasta,8.795083,2.221381,58.524254,284.127072
7,Dairy and Egg Products,14.106148,13.313498,11.963781,223.717314
8,Fast Foods,12.118678,12.544628,23.111928,252.534435
9,Fats and Oils,0.407818,67.810091,5.346318,623.695455


In [88]:
fig = px.bar(macros_by_group,x="group",y=["protein","fat","carbs"],facet_col="variable",width=900,height=800,title="Average protein,fat,and carbs per food group (g)")

In [89]:
fig2 = px.bar(macros_by_group,x="group",y="calories",title="Average calories per food group")

In [90]:
fig.show()

In [91]:
fig2.show()